In [ ]:
import os
import pandas as pd

import numpy as np

import tensorflow as tf
import tensorflow_addons as tfa

from tensorflow import keras

from tensorflow.keras import layers
from tensorflow.keras import models

from tensorflow.keras.models import load_model
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor

from keras.constraints import maxnorm

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn import pipeline

from scipy.stats import shapiro, normaltest, anderson
from statsmodels.stats.diagnostic import lilliefors

from joblib import load

import seaborn as sns
from statsmodels.graphics.gofplots import qqplot
import matplotlib.pyplot as plt

In [ ]:
X_treinamento = pd.read_json(r'dados/precipitacao/X_treinamento_80.json')
X_teste = pd.read_json      (r'dados/precipitacao/X_teste_20.json')
treino_json = pd.read_json  (r'dados/precipitacao/y_treinamento_80.json', typ='series')
teste_json = pd.read_json   (r'dados/precipitacao/y_teste_20.json', typ='series')

In [ ]:
a = []
b = []

for i in range(treino_json.size):
    a.append(list(treino_json[i]))
    
for i in range(teste_json.size):
    b.append(list(teste_json[i]))

In [ ]:
y_treinamento = np.array(a, dtype='float32')
y_teste = np.array(b, dtype='float32')

In [ ]:
def build_model(
    optimizer,
    learning_rate,
    activation,
    init_mode,
    dropout_rate,
    weight_constraint,
    neurons,
    regularizers
):
    model = models.Sequential()
    
    # layer 1
    model.add(
        layers.Dense(
            neurons,
            activation = activation,
            kernel_initializer=init_mode,
            kernel_constraint=maxnorm(weight_constraint),
            kernel_regularizer=regularizers,
            input_shape=[len(X_treinamento.keys())]
        )
    )
    model.add(layers.Dropout(dropout_rate))
    
    # layer 2
    model.add(layers.Dense(256))
    
    model.compile(
        loss='mse',
        optimizer=optimizer(learning_rate),
        metrics=['mae', 'mse', keras.metrics.RootMeanSquaredError()]
    )
    
    return model

In [ ]:
loaded_model = KerasRegressor(build_fn=build_model, verbose=1)

In [ ]:
# loaded_model.model = load_model(
#     r'resultados/model.h5',
#     custom_objects={'mish': tfa.activations.mish}
# )

In [ ]:
loaded_model.model = load_model(r'resultados/model.h5')

In [ ]:
scaler = load(r'resultados/scaler.joblib')

In [ ]:
loaded_steps = [('scaler', scaler), 
         ('estimator', loaded_model)]

In [ ]:
loaded_pipe = pipeline.Pipeline(loaded_steps)

In [ ]:
plm = loaded_pipe.predict(X_teste)

In [ ]:
expectedWlm = []
for i in range(y_teste.shape[0]):
    dfE = pd.DataFrame(
        y_teste[i].reshape(16,16), 
        columns=['0', '1', '2', '3', '4', '5', '6', '7', '8', 
                 '9', '10', '11', '12', '13', '14', '15'])
    w = dfE.values.sum()
    expectedWlm.append(w)

In [ ]:
predictedWlm = []
for i in range(plm.shape[0]):
    dfP = pd.DataFrame(
        plm[i].reshape(16,16), 
        columns=['0', '1', '2', '3', '4', '5', '6', '7', '8', 
                 '9', '10', '11', '12', '13', '14', '15'])
#     for i in range(16):
#         dfP.loc[dfP[str(i)] < 0, str(i)] = 0
    w = dfP.values.sum()
    predictedWlm.append(w)

In [ ]:
for prediction, real in zip(predictedWlm, expectedWlm):
    print("Prediction: %f - Real: %f - Error: %f" % (prediction, real, real - prediction))

In [ ]:
print("{0:0.5f}".format(mean_absolute_error(predictedWlm, expectedWlm)))

In [ ]:
print("{0:0.5f}".format(mean_squared_error(predictedWlm, expectedWlm)))

In [ ]:
print("{0:0.5f}".format(mean_squared_error(predictedWlm, expectedWlm, squared=False)))

In [ ]:
print(r2_score(predictedWlm, expectedWlm))

In [ ]:
hist = pd.read_json(r'resultados/history.json')

In [ ]:
sns.set_theme()

In [ ]:
def plot_history(hist):
#     hist = pd.DataFrame(grid_result.best_estimator_['estimator'].model.history.history)
    hist.reset_index(inplace=True)
    hist = hist.rename(columns = {'index':'epoch'})

    plt.figure()
    plt.xlabel('Época')
    plt.ylabel('Erro [MSE]')
    plt.plot(hist['epoch'], hist['mse'],
           label='Erro')
    plt.ylim([0,2])
    plt.legend()


plot_history(hist)

In [ ]:
error = pd.Series(np.array(expectedWlm) - np.array(predictedWlm))

In [ ]:
error.to_json(r'resultados/error.json')

In [ ]:
plt.hist(error, bins = 15)
plt.xlabel("Resíduos")
_ = plt.ylabel("Observações")

In [ ]:
resultados = pd.DataFrame({'Valores Preditos':predictedWlm, 'Resíduos':error})

In [ ]:
sns.scatterplot(data=resultados, x="Valores Preditos", y="Resíduos")
ax = sns.lineplot(x=resultados['Valores Preditos'],y=0, dashes=[(2, 2), (2, 2)])
ax.lines[0].set_linestyle("--")
plt.show()

In [ ]:
qqplot(error, line='s')
plt.show()

In [ ]:
sp = shapiro(error)
sp

In [ ]:
nt = normaltest(error)
nt

In [ ]:
lf = lilliefors(error)
lf

In [ ]:
result = anderson(error)
print('Statistic: %.3f' % result.statistic)
for i in range(len(result.critical_values)):
    sl, cv = result.significance_level[i], result.critical_values[i]
    if result.statistic < result.critical_values[i]:
        print('%.3f: %.3f, data looks normal (fail to reject H0)' % (sl, cv))
    else:
        print('%.3f: %.3f, data does not look normal (reject H0)' % (sl, cv))